In [1]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, ForeignKey
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String, Float
import pandas as pd
engine = create_engine('sqlite:///cranes.sqlite')
metadata = MetaData()
# define the tables
museum = Table('museum', metadata,
              Column('name', String),
              Column('code', String, primary_key = True),
              Column('contact',String),
              Column('email', String),
              )
taxonomy = Table('taxonomy', metadata,
              Column('name', String, primary_key = True),
              Column('kingdom', String),
              Column('phylum', String),
              Column('class', String),
              Column('order', String),
              Column('family', String),
              Column('genus', String),
              Column('specific_epithet', String),
              )
specimen = Table('specimen', metadata,
              Column('ID', Integer, primary_key = True),
              Column('scientific_name', String, ForeignKey("taxonomy.name")),
              Column('museum_code', String, ForeignKey("museum.code")),
              Column('sex', String),
              Column('lifestage', String),
              Column('mass', Float),
              Column('country_code', String, ForeignKey("country.code")),
              Column('state', String),
              Column('county', String),
              Column('locality', String),
              Column('elevation', Float),
              Column('date', String),
              )
country = Table('country', metadata,
              Column('name', String),
              Column('code', String, primary_key = True),
              Column('continent', String),
              )
metadata.create_all(engine)

In [2]:
# I have 4 tables, so I only loaded data of 2 tables for practice
# load museum table into database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///cranes.sqlite')
conn = engine.connect()
data = pd.read_csv('Documents/cranes.csv',encoding = 'Latin-1',usecols=['museum','museum_code','dataset_contact','dataset_contact_email'])
info=data.drop_duplicates(keep="first")
i=0
while True:
    ins = museum.insert().values(name=info['museum'].tolist()[i],code= info['museum_code'].tolist()[i],contact=info['dataset_contact'].tolist()[i],email=info['dataset_contact_email'].tolist()[i])
    conn.execute(ins)
    i=i+1
    if i == len(info['museum'].tolist()):
        print('done!')
        break

done!


In [5]:
# load specimen table into database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///cranes.sqlite')
conn = engine.connect()
data = pd.read_csv('Documents/cranes.csv',encoding = 'Latin-1',usecols=['catalogID','museum_code','scientific_name','sex','lifestage','mass','date','country_code','state','county','locality','elevation'])
i=0
while True:
    ins = specimen.insert().values(ID=data['catalogID'].tolist()[i],scientific_name= data['scientific_name'].tolist()[i],museum_code=data['museum_code'].tolist()[i],sex=data['sex'].tolist()[i],lifestage=data['lifestage'].tolist()[i],mass=data['mass'].tolist()[i],date=data['date'].tolist()[i],country_code=data['country_code'].tolist()[i],state=data['state'].tolist()[i],county=data['county'].tolist()[i],locality=data['locality'].tolist()[i],elevation=data['elevation'].tolist()[i])
    conn.execute(ins)
    i=i+1
    if i == len(data['catalogID'].tolist()):
        print('done!')
        break

done!


In [8]:
# queries for testing
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import sql, select, join, desc

engine = create_engine('sqlite:///cranes.sqlite')
conn = engine.connect()
DBInfo = MetaData(engine)
museum = Table('museum', DBInfo, autoload=True)
specimen = Table('specimen', DBInfo, autoload=True)

query1 = select([museum.c.contact,museum.c.email]).where(museum.c.code == 'MSB')
result1 = conn.execute(query1)
for row in result1:
    print(row)

query2 = select([specimen.c.scientific_name, specimen.c.sex,specimen.c.country_code]).where(specimen.c.museum_code == 'UF')
result2 = conn.execute(query2)
for row in result2:
    print(row)

('Christopher Witt', 'cwitt@unm.edu')
('Grus americana', 'male', 'USA')
('Grus americana', 'female', 'USA')
